# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [2]:
poly_modulus = 64
bit_sizes= [40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
print(ctx.param.coeff_modulus)

AttributeError: 'Context' object has no attribute 'param'

In [4]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[0, 0, 1, 0, 1099511623296, 0, 0, 1099511623296, 1, 1099511623296, 0, 1, 1, 0, 1, 1, 1, 1099511623296, 1099511623296, 0, 1099511623296, 1099511623296, 1, 0, 1, 1099511623296, 1099511623296, 0, 0, 1099511623296, 0, 0, 1099511623296, 1, 1099511623296, 1099511623296, 1099511623296, 1, 0, 1, 0, 1, 1099511623296, 1, 1, 1099511623296, 1099511623296, 1099511623296, 1, 0, 0, 1099511623296, 0, 1099511623296, 0, 1099511623296, 0, 1099511623296, 1, 1099511623296, 1, 0, 0, 0]]


In [5]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [6]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[1, 0, 0, 1]     [0, 0, 1]     [0, 0, 1]


### Decodes back to Integer

In [7]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

9
4
4


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [8]:
encrypter = Encryptor(ctx, pk)

In [9]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [10]:
decrypter = Decryptor(ctx, sk)

In [11]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [12]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

9     4     4


## Evaluator

In [13]:
eval = Evaluator(ctx)

In [14]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

13


In [15]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

13


In [16]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

13


### Verify result

In [17]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [18]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[987087971196, 509284154373, 721405037536, 772102664538, 1030088083642, 1052905798122, 1093025457831, 947769491175, 618058815223, 1072046300069, 554087671929, 144118039165, 380716416165, 575355614646, 469022030028, 357329678618, 944420411006, 357763776412, 563074906213, 473535859797, 1088992316867, 899911537188, 609962850961, 678371846628, 12425926942, 867269115907, 460748089393, 45403017622, 446587417768, 773046172482, 12063429463, 479064392116, 1021522144736, 102929326589, 334170319241, 616660253790, 929570670310, 244728605527, 4594639400, 775101949762, 650638079789, 25894757258, 1094352104068, 899152637187, 607147469352, 513111635530, 500860647414, 78645883275, 15909411353, 438507837827, 731705859641, 67569947813, 824265226989, 38913074303, 901761114183, 426838167903, 897632236849, 937017220407, 617676275857, 570551837726, 488164072052, 819085274516, 15861615034, 804773831349]], [[189092571812, 481560639515, 344227186144, 987363810518, 266715789464, 53542094113, 5177132122

In [19]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

36      36


In [20]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
context = Context(EncryptionParams(128, CoeffModulus().create(128, [30]), 128))

In [21]:
keys = KeyGenerator(context).keygen()

In [22]:
x = th.Tensor([[1,2,3],[4,5,6]]).encrypt(protocol="bfv", private_key=keys[0], context = context)

In [23]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x127976f70>

In [24]:
x.decrypt(private_key = keys[0])

tensor([[1, 2, 3],
        [4, 5, 6]])